<a href="https://colab.research.google.com/github/haroldoproenca/lab_datascience/blob/main/GeracaoMassaDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ESTUDO DE CASO: Amana Beauty Lab**
##### **Nome**: Haroldo Paes de Proença
##### **Case**: *Geração de base de dados para Varejo para fim de estudos* <br> **Base de Dados**: <br>*produtos.csv* <br> *filiais.csv* <br> *clientes.csv* <br> *estoque.csv* <br> *vendas.csv*

--------------

## **INSTALAÇÃO/CARREGAMENTO DE BIBLIOTECAS**

---



In [ ]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta
from google.colab import drive
import os

### **MONTANDO GOOGLE DRIVE**

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


### **DIRETÓRIO ONDE SERÃO GRAVADOS OS DATASETS E CÓDIGOS UTILIZADOS**

In [ ]:
DRIVE_PATH = '/content/drive/MyDrive/lab_datascience'
if not os.path.exists(DRIVE_PATH):
    os.makedirs(DRIVE_PATH)

### **PASSO 1: GERAÇÃO DA TABELA DE PRODUTOS**
Aqui, vamos criar os SKUs. Usaremos as informações de contexto da "Amana Beauty Lab" (Linha Poranga, Essencial, etc.) para mapear os tiers "Alto Padrão" (30%) e "Médio Padrão" (70%). Os nomes serão gerados com base nos ativos brasileiros.

#### **TOTAL DE SKUs A SER GERADOS**

In [ ]:
total_skus=2000

In [ ]:
def gerar_produtos(total_skus):
    # Categorias e suas proporções de mercado (estimativa)
    categorias_map = {
        'Shampoo': 0.15,
        'Condicionador': 0.15,
        'Sabonete em barra': 0.15,
        'Sabonete liquido para maos': 0.10,
        'Hidratante para os corpos': 0.15,
        'Hidratante para as maos': 0.10,
        'Perfume': 0.10,
        'Colonia': 0.10
    }

    # Descrições e custos base por categoria
    descricoes_map = {
        'Shampoo': ['250ml', '300ml', '500ml'],
        'Condicionador': ['250ml', '300ml', '500ml'],
        'Sabonete em barra': ['90g', '120g', 'Caixa 3x90g'],
        'Sabonete liquido para maos': ['200ml', '350ml Refil'],
        'Hidratante para os corpos': ['200g', '400ml Refil'],
        'Hidratante para as maos': ['50g', '75g'],
        'Perfume': ['50ml Eau de Parfum', '100ml Eau de Parfum'],
        'Colonia': ['100ml Body Splash', '200ml Água de Colonia']
    }

    custo_base_map = {
        'Shampoo': 12.0,
        'Condicionador': 13.0,
        'Sabonete em barra': 4.0,
        'Sabonete liquido para maos': 9.0,
        'Hidratante para os corpos': 15.0,
        'Hidratante para as maos': 8.0,
        'Perfume': 45.0,
        'Colonia': 25.0
    }

    # Listas para geração de nomes (baseado no contexto da Amana)
    ingredientes_amz = ['Açaí', 'Cupuaçu', 'Murumuru', 'Andiroba', 'Castanha', 'Buriti', 'Maracujá', 'Cacau', 'Pataua']
    temas_br = ['Poranga', 'Curumim', 'Raízes', 'Essencial', 'Bossa', 'Samba', 'Tupi']
    temas_chas = ['Erva-Cidreira', 'Camomila', 'Capim-Limão', 'Hortelã', 'Jasmim']
    formatos_premium = ['Elixir', 'Sérum', 'Óleo Bruto', 'Manteiga', 'Bruma']

    produtos = []

    for _ in range(total_skus):
        # 1. Definir Categoria, Tier e EAN
        ean = '789' + str(random.randint(10**9, 10**10 - 1))
        categoria = np.random.choice(list(categorias_map.keys()), p=list(categorias_map.values()))
        tier = np.random.choice(['Alto Padrão', 'Médio Padrão'], p=[0.3, 0.7])

        # 2. Gerar Nome e Descrição
        descr = random.choice(descricoes_map[categoria])
        nome = ""
        if tier == 'Alto Padrão':
            # Mapeando para Linha Poranga ou Raízes
            if categoria in ['Perfume', 'Hidratante para os corpos', 'Hidratante para as maos']:
                nome = f"{random.choice(formatos_premium)} {random.choice(temas_br)} de {random.choice(ingredientes_amz)}"
            else:
                nome = f"{categoria} Premium {random.choice(temas_br)} {random.choice(ingredientes_amz)}"
        else:
            # Mapeando para Linha Essencial ou Curumim
            nome = f"{categoria} {random.choice(temas_chas)} e {random.choice(ingredientes_amz)}"

        # 3. Gerar Custos e Preços
        custo_base = custo_base_map[categoria]
        # Multiplicador de tier (Alto padrão é mais caro) e volatilidade
        multiplicador_tier = 1.8 if tier == 'Alto Padrão' else 1.0
        custo_produto = round(custo_base * multiplicador_tier * np.random.uniform(0.9, 1.1), 2)

        # 4. Calcular colunas financeiras (seguindo suas regras)
        lucro_minimo = round(custo_produto * 1.0, 2)
        lucro_maximo = round(custo_produto * 2.0, 2)

        # O lucro real (para o preço de lista) será um valor entre o min e o max
        lucro_escolhido = round(np.random.uniform(lucro_minimo, lucro_maximo), 2)

        valor_liquido = round(custo_produto + lucro_escolhido, 2)
        impostos = round(valor_liquido * 0.20, 2)
        valor_bruto = round(valor_liquido + impostos, 2)

        produtos.append({
            'COD_EAN': ean,
            'NOME_PRODUTO': nome,
            'DESCR_PRODUTO': descr,
            'CATEGORIA': categoria, # Coluna extra para lógica
            'TIER': tier, # Coluna extra para lógica
            'CUSTO_PRODUTO': custo_produto,
            'LUCRO_MINIMO': lucro_minimo,
            'LUCRO_MAXIMO': lucro_maximo,
            'VALOR_LIQUIDO': valor_liquido, # Preço de lista (base)
            'IMPOSTOS': impostos,
            'VALOR_BRUTO': valor_bruto # Preço de lista (base)
        })

    df_produtos = pd.DataFrame(produtos)
    print("Tabela de Produtos gerada com sucesso.")
    return df_produtos

# Gerar e salvar
df_produtos = gerar_produtos(total_skus=2000)
df_produtos.to_csv(f'{DRIVE_PATH}/produtos.csv', index=False, encoding='utf-8-sig')
print(f"Arquivo 'produtos.csv' salvo em {DRIVE_PATH}")

Tabela de Produtos gerada com sucesso.
Arquivo 'produtos.csv' salvo em /content/drive/MyDrive/lab_datascience


In [ ]:
df_produtos.head()

,COD_EAN,NOME_PRODUTO,DESCR_PRODUTO,CATEGORIA,TIER,CUSTO_PRODUTO,LUCRO_MINIMO,LUCRO_MAXIMO,VALOR_LIQUIDO,IMPOSTOS,VALOR_BRUTO
0,7892297602191,Sabonete liquido para maos Hortelã e Cupuaçu,200ml,Sabonete liquido para maos,Médio Padrão,9.68,9.68,19.36,27.20,5.44,32.64
1,7892563224432,Sabonete liquido para maos Jasmim e Cacau,350ml Refil,Sabonete liquido para maos,Médio Padrão,8.53,8.53,17.06,23.85,4.77,28.62
2,7897080934366,Sabonete liquido para maos Capim-Limão e Murumuru,200ml,Sabonete liquido para maos,Médio Padrão,9.12,9.12,18.24,20.28,4.06,24.34
3,7899332555966,Perfume Jasmim e Murumuru,100ml Eau de Parfum,Perfume,Médio Padrão,41.92,41.92,83.84,120.01,24.00,144.01
4,7898834922236,Perfume Jasmim e Murumuru,50ml Eau de Parfum,Perfume,Médio Padrão,42.34,42.34,84.68,86.50,17.30,103.80


### **PASSO 2: GERAÇÃO DA TABELA DE FILIAIS**
Agora, vamos gerar as filiais físicas (lojas) com base no parâmetro num_filiais_fisicas e adicionar o E-commerce, que sempre existe.

In [ ]:
num_filiais_fisicas=6

In [ ]:
def gerar_filiais(num_filiais_fisicas):
    print(f"Gerando {num_filiais_fisicas} filiais físicas + 1 E-commerce...")
    fake = Faker('pt_BR')
    filiais = []

    # 1. Definir perfis das lojas físicas
    n_alto = num_filiais_fisicas // 3
    n_medio = num_filiais_fisicas // 3
    n_baixo = num_filiais_fisicas - n_alto - n_medio # Garante que o total feche

    perfis = (['Alto Padrão'] * n_alto) + (['Médio Padrão'] * n_medio) + (['Baixo Padrão'] * n_baixo)
    random.shuffle(perfis)

    # 2. Criar lojas físicas
    for i, perfil in enumerate(perfis):
        cod_filial = f'LOJA_{i+1:03d}'
        nome_filial = f'Amana Lab - {fake.street_name()}'
        if perfil == 'Alto Padrão':
            nome_filial = f'Amana Lab - Shopping {fake.last_name()}'

        filiais.append({
            'COD_FILIAL': cod_filial,
            'NOME_FILIAL': nome_filial,
            'PERFIL': perfil
        })

    # 3. Adicionar E-commerce
    filiais.append({
        'COD_FILIAL': 'ECOM_001',
        'NOME_FILIAL': 'Amana Lab - E-commerce',
        'PERFIL': 'ECOMMERCE'
    })

    df_filiais = pd.DataFrame(filiais)
    print("Tabela de Filiais gerada com sucesso.")
    return df_filiais

# Gerar e salvar (Exemplo com 6 lojas físicas, totalizando 7 filiais com o e-com)
df_filiais = gerar_filiais(num_filiais_fisicas=6)
df_filiais.to_csv(f'{DRIVE_PATH}/filiais.csv', index=False, encoding='utf-8-sig')
print(f"Arquivo 'filiais.csv' salvo em {DRIVE_PATH}")


Gerando 6 filiais físicas + 1 E-commerce...
Tabela de Filiais gerada com sucesso.
Arquivo 'filiais.csv' salvo em /content/drive/MyDrive/lab_datascience


,COD_FILIAL,NOME_FILIAL,PERFIL
0,LOJA_001,Amana Lab - Shopping Ribeiro,Alto Padrão
1,LOJA_002,Amana Lab - Avenida Isaque Pereira,Médio Padrão
2,LOJA_003,Amana Lab - Shopping Leão,Alto Padrão
3,LOJA_004,Amana Lab - Parque Stella Caldeira,Médio Padrão
4,LOJA_005,Amana Lab - Lagoa Pimenta,Baixo Padrão
5,LOJA_006,Amana Lab - Aeroporto Teixeira,Baixo Padrão
6,ECOM_001,Amana Lab - E-commerce,ECOMMERCE


In [ ]:
df_filiais.head()

,COD_FILIAL,NOME_FILIAL,PERFIL
0,LOJA_001,Amana Lab - Shopping Ribeiro,Alto Padrão
1,LOJA_002,Amana Lab - Avenida Isaque Pereira,Médio Padrão
2,LOJA_003,Amana Lab - Shopping Leão,Alto Padrão
3,LOJA_004,Amana Lab - Parque Stella Caldeira,Médio Padrão
4,LOJA_005,Amana Lab - Lagoa Pimenta,Baixo Padrão


### **PASSO 3 - GERAÇÃO DE TABELAS DE CLIENTES**
Vamos criar os clientes com perfis de compra que influenciarão suas escolhas de produtos.

In [ ]:
num_clientes=3000

In [ ]:
def gerar_clientes(num_clientes):
    print(f"Gerando {num_clientes} clientes...")
    fake = Faker('pt_BR')
    clientes = []

    # Perfis de compra (baseado em valor/frequência)
    perfis_compra_map = {
        'Premium': 0.20,         # Compra Alto Padrão, alta frequência/valor
        'Custo-Benefício': 0.50, # Compra Médio Padrão, média frequência
        'Ocasional': 0.20,       # Compra esporadicamente
        'Caçador de Ofertas': 0.10 # Foco em desconto
    }

    for i in range(num_clientes):
        cod_cliente = f'CLI_{i+1:05d}'
        nome = fake.first_name()
        sobrenome = fake.last_name()
        perfil = np.random.choice(list(perfis_compra_map.keys()), p=list(perfis_compra_map.values()))

        clientes.append({
            'COD_CLIENTE': cod_cliente,
            'NOME_CLIENTE': nome,
            'SOBRENOME_CLIENTE': sobrenome,
            'PERFIL_COMPRA': perfil
        })

    df_clientes = pd.DataFrame(clientes)
    print("Tabela de Clientes gerada com sucesso.")
    return df_clientes

# Gerar e salvar
df_clientes = gerar_clientes(num_clientes=3000)
df_clientes.to_csv(f'{DRIVE_PATH}/clientes.csv', index=False, encoding='utf-8-sig')
print(f"Arquivo 'clientes.csv' salvo em {DRIVE_PATH}")

Gerando 3000 clientes...
Tabela de Clientes gerada com sucesso.
Arquivo 'clientes.csv' salvo em /content/drive/MyDrive/lab_datascience


In [ ]:
df_clientes.head()

,COD_CLIENTE,NOME_CLIENTE,SOBRENOME_CLIENTE,PERFIL_COMPRA
0,CLI_00001,Ana Sophia,Siqueira,Custo-Benefício
1,CLI_00002,André,Peixoto,Custo-Benefício
2,CLI_00003,Laura,Pastor,Ocasional
3,CLI_00004,Cecilia,Moura,Custo-Benefício
4,CLI_00005,Théo,Sales,Ocasional


### **PASSO 4: GERAÇÃO DA TABELA DE ESTOQUE**
E tabela conterá os parâmetros de estoque (Mínimo, Máximo) e uma "foto" do estoque atual. É foto instantânea (snapshot) do estoque em uma data específica, que já é muito útil para problemas de abastecimento.

In [ ]:
# PASSO 4: GERAR TABELA DE ESTOQUE (SNAPSHOT)

def gerar_estoque_snapshot(df_filiais, df_produtos, data_foto):
    print(f"Gerando snapshot de estoque para {data_foto.date()}...")

    # Criar todas as combinações de filial x produto
    combinacoes = pd.MultiIndex.from_product(
        [df_filiais['COD_FILIAL'], df_produtos['COD_EAN']],
        names=['COD_FILIAL', 'EAN']
    ).to_frame(index=False)

    # Definir parâmetros de estoque
    combinacoes['ESTOQUE_MINIMO'] = np.random.randint(10, 50, size=len(combinacoes))
    combinacoes['ESTOQUE_MAXIMO'] = combinacoes['ESTOQUE_MINIMO'] * np.random.randint(3, 6, size=len(combinacoes))

    # Gerar estoque atual com volatilidade
    # (pode estar abaixo do mínimo ou acima do máximo para simular recebimentos/rupturas)
    combinacoes['ESTOQUE_ATUAL'] = np.random.randint(
        combinacoes['ESTOQUE_MINIMO'] - 10,
        combinacoes['ESTOQUE_MAXIMO'] + 20,
        size=len(combinacoes)
    )
    # Garantir que o estoque não seja negativo
    combinacoes['ESTOQUE_ATUAL'] = combinacoes['ESTOQUE_ATUAL'].clip(lower=0)

    combinacoes['DATA_FOTO_ESTOQUE'] = data_foto.date()

    print("Tabela de Estoque (snapshot) gerada com sucesso.")
    return combinacoes

# Gerar e salvar (Usando a data final do período de vendas como data da foto)
data_inicio_vendas = datetime(2022, 1, 1)
data_fim_vendas = datetime(2026, 1, 1)

df_estoque = gerar_estoque_snapshot(df_filiais, df_produtos, data_fim_vendas)
df_estoque.to_csv(f'{DRIVE_PATH}/estoque.csv', index=False, encoding='utf-8-sig')
print(f"Arquivo 'estoque.csv' salvo em {DRIVE_PATH}")

Gerando snapshot de estoque para 2026-01-01...
Tabela de Estoque (snapshot) gerada com sucesso.
Arquivo 'estoque.csv' salvo em /content/drive/MyDrive/lab_datascience


In [ ]:
df_estoque.head()

,COD_FILIAL,EAN,ESTOQUE_MINIMO,ESTOQUE_MAXIMO,ESTOQUE_ATUAL,DATA_FOTO_ESTOQUE
0,LOJA_001,7892297602191,19,76,36,2026-01-01
1,LOJA_001,7892563224432,33,99,24,2026-01-01
2,LOJA_001,7897080934366,33,132,145,2026-01-01
3,LOJA_001,7899332555966,23,69,73,2026-01-01
4,LOJA_001,7898834922236,25,75,53,2026-01-01


### **PASSO 5 - GERAÇÃO DE TABELA DE VENDAS (TRANSACIONAL)**
Esta é a etapa mais complexa. Vamos gerar as linhas de nota fiscal. O número de vendas que você passa como parâmetro será o número de notas fiscais (transações), e cada nota terá um número variável de itens (linhas).

Isso irá simular:

Sazonalidade: Picos no Natal, Black Friday, Dia das Mães, etc.

Afinidade Perfil-Produto: Lojas de "Alto Padrão" vendem mais produtos "Alto Padrão". Clientes "Premium" compram mais produtos "Alto Padrão".

Descontos: "Caçadores de Ofertas" e a Black Friday terão mais descontos.

In [ ]:
# PASSO 5: GERAR TABELA DE VENDAS (VERSÃO ATUALIZADA)

def gerar_vendas(num_notas, data_inicio, data_fim, df_produtos, df_filiais, df_clientes):
    print(f"Iniciando geração de {num_notas} notas fiscais (com múltiplos itens)...")

    # --- 5.1: Preparar dados de afinidade ---
    produtos_alto = df_produtos[df_produtos['TIER'] == 'Alto Padrão']['COD_EAN'].tolist()
    produtos_medio = df_produtos[df_produtos['TIER'] == 'Médio Padrão']['COD_EAN'].tolist()

    # Probabilidade de comprar 'Alto Padrão' baseado no perfil da loja
    prob_loja_map = {
        'Alto Padrão': 0.70,
        'Médio Padrão': 0.20,
        'Baixo Padrão': 0.05,
        'ECOMMERCE': 0.30  # Média do mix
    }

    # Probabilidade de comprar 'Alto Padrão' baseado no perfil do cliente
    prob_cliente_map = {
        'Premium': 0.80,
        'Custo-Benefício': 0.15,
        'Ocasional': 0.20,
        'Caçador de Ofertas': 0.10
    }

    # --- 5.2: Gerar Datas com Sazonalidade ---
    datas_pool = pd.date_range(data_inicio, data_fim)
    pesos = np.ones(len(datas_pool))

    # Picos de sazonalidade (aumenta o peso desses dias)
    for i, data in enumerate(datas_pool):
        # Dia das Mães (Maio)
        if data.month == 5 and data.day > 5 and data.day < 15:
            pesos[i] *= 3
        # Dia dos Namorados (Junho)
        if data.month == 6 and data.day > 2 and data.day < 13:
            pesos[i] *= 3
        # Dia dos Pais (Agosto)
        if data.month == 8 and data.day > 4 and data.day < 14:
            pesos[i] *= 2
        # Black Friday (Novembro)
        if data.month == 11 and data.day > 20 and data.day < 30:
            pesos[i] *= 6
        # Natal (Dezembro)
        if data.month == 12 and data.day > 10 and data.day < 25:
            pesos[i] *= 7

    pesos_normalizados = pesos / pesos.sum()

    # --- 5.3: Gerar as Notas Fiscais (Cabeçalho) ---
    print(f"Gerando cabeçalhos de {num_notas} notas...")
    df_notas = pd.DataFrame({
        'NUM_NOTA': range(100001, 100001 + num_notas),
        'SERIE_NOTA': '001',
        'DATA_NOTA': np.random.choice(datas_pool, size=num_notas, p=pesos_normalizados),
        'COD_CLIENTE': np.random.choice(df_clientes['COD_CLIENTE'], size=num_notas),
        'COD_FILIAL': np.random.choice(df_filiais['COD_FILIAL'], size=num_notas)
    })

    # --- 5.4: Gerar os Itens da Nota (Explodir) ---
    # Cada nota terá de 1 a 5 itens (média ~2.5)
    num_itens_por_nota = np.random.poisson(1.5, num_notas) + 1

    # Repetir as linhas da nota fiscal para cada item
    df_vendas = df_notas.loc[df_notas.index.repeat(num_itens_por_nota)].reset_index(drop=True)
    total_linhas = len(df_vendas)
    print(f"Total de {total_linhas} linhas de itens geradas.")

    # --- 5.5: Enriquecer Itens (Produto, Preço, Desconto) ---
    print("Enriquecendo linhas com produtos e preços...")

    # Merge dos perfis de cliente e filial
    df_vendas = df_vendas.merge(df_clientes[['COD_CLIENTE', 'PERFIL_COMPRA']], on='COD_CLIENTE')
    df_vendas = df_vendas.merge(df_filiais[['COD_FILIAL', 'PERFIL']], on='COD_FILIAL')

    # 1. Escolher o TIER do produto (Alto vs Médio Padrão)
    prob_alto_loja = df_vendas['PERFIL'].map(prob_loja_map)
    prob_alto_cliente = df_vendas['PERFIL_COMPRA'].map(prob_cliente_map)
    # Média das probabilidades do cliente e da loja
    prob_final_alto = (prob_alto_loja + prob_alto_cliente) / 2

    rand_tier = np.random.rand(total_linhas)
    df_vendas['TIER_COMPRA'] = np.where(rand_tier < prob_final_alto, 'Alto Padrão', 'Médio Padrão')

    # 2. Escolher o EAN específico
    picks_alto = np.random.choice(produtos_alto, size=total_linhas)
    picks_medio = np.random.choice(produtos_medio, size=total_linhas)
    df_vendas['COD_EAN'] = np.where(df_vendas['TIER_COMPRA'] == 'Alto Padrão', picks_alto, picks_medio)

    # 3. Trazer dados do produto (preço de lista, custo)
    df_vendas = df_vendas.merge(
        df_produtos[['COD_EAN', 'CUSTO_PRODUTO', 'VALOR_LIQUIDO', 'VALOR_BRUTO']],
        on='COD_EAN'
    )

    # Renomeação explícita
    df_vendas.rename(columns={
        'VALOR_LIQUIDO': 'VALOR_LIQUIDO_lista', # Este é o preço de tabela unitário
        'VALOR_BRUTO': 'VALOR_BRUTO_lista'   # Este é o preço de tabela bruto unitário
    }, inplace=True)

    # 4. Calcular Quantidade
    df_vendas['QUANTIDADE'] = np.random.poisson(0.3, total_linhas) + 1 # Maioria compra 1 unidade

    # 5. Calcular Desconto
    # Probabilidade base de desconto
    prob_desconto = np.full(total_linhas, 0.05)
    # Aumenta prob para Caçador de Ofertas
    prob_desconto = np.where(df_vendas['PERFIL_COMPRA'] == 'Caçador de Ofertas', 0.4, prob_desconto)
    # Aumenta prob na Black Friday
    prob_desconto = np.where((df_vendas['DATA_NOTA'].dt.month == 11) & (df_vendas['DATA_NOTA'].dt.day > 20), 0.7, prob_desconto)

    tem_desconto = np.random.rand(total_linhas) < prob_desconto
    pct_desconto = np.random.uniform(0.1, 0.35, total_linhas) # Desconto de 10% a 35%
    df_vendas['DESCONTO_PCT'] = np.where(tem_desconto, pct_desconto, 0)

    # 6. Calcular Valores Finais da Transação
    df_vendas['VALOR_LIQUIDO_UNITARIO_CALC'] = df_vendas['VALOR_LIQUIDO_lista'] * (1 - df_vendas['DESCONTO_PCT'])

    # Garantir que o desconto nunca seja menor que o custo
    df_vendas['VALOR_LIQUIDO_UNITARIO_FINAL'] = df_vendas[['VALOR_LIQUIDO_UNITARIO_CALC', 'CUSTO_PRODUTO']].max(axis=1)

    # ***** INÍCIO DAS ALTERAÇÕES *****
    # (Calculando os valores totais da linha da nota)

    # VALOR_LIQUIDO (Final, pós-desconto)
    df_vendas['VALOR_LIQUIDO'] = round(df_vendas['VALOR_LIQUIDO_UNITARIO_FINAL'] * df_vendas['QUANTIDADE'], 2)

    # IMPOSTOS (Calculado sobre o VALOR_LIQUIDO final)
    df_vendas['IMPOSTOS'] = round(df_vendas['VALOR_LIQUIDO'] * 0.20, 2)

    # VALOR_BRUTO (Final, com impostos)
    df_vendas['VALOR_BRUTO'] = round(df_vendas['VALOR_LIQUIDO'] + df_vendas['IMPOSTOS'], 2)

    # DESCONTO (Valor monetário do desconto aplicado)
    # (Preço de lista total - Preço líquido final)
    valor_lista_total = df_vendas['VALOR_LIQUIDO_lista'] * df_vendas['QUANTIDADE']
    df_vendas['DESCONTO'] = round(valor_lista_total - df_vendas['VALOR_LIQUIDO'], 2)

    # ***** FIM DAS ALTERAÇÕES *****

    # Selecionar colunas finais
    # (Adicionando 'VALOR_LIQUIDO_lista' e 'IMPOSTOS' e reordenando)
    colunas_finais = [
        'NUM_NOTA', 'SERIE_NOTA', 'DATA_NOTA', 'COD_FILIAL', 'COD_CLIENTE',
        'COD_EAN', 'QUANTIDADE',
        'VALOR_LIQUIDO_lista', # Preço unitário de tabela (antes do desconto)
        'DESCONTO',            # Valor total do desconto na linha
        'VALOR_LIQUIDO',       # Valor total da linha (pós-desconto, antes de imposto)
        'IMPOSTOS',            # Valor total do imposto na linha
        'VALOR_BRUTO'          # Valor total da linha (pós-desconto, com imposto)
    ]

    # Precisamos renomear a 'VALOR_LIQUIDO_lista' pois ela já está com o nome correto
    # Mas ela é um valor unitário, vamos manter assim como referência de preço

    df_vendas_final = df_vendas[colunas_finais]

    print("Tabela de Vendas gerada com sucesso.")
    return df_vendas_final

In [ ]:
PARAM_NUM_NOTAS = 150000
PARAM_DATA_INICIO = datetime(2022, 1, 1)
PARAM_DATA_FIM = datetime(2026, 1, 1)

In [ ]:
print(f"Iniciando a geração de {PARAM_NUM_NOTAS} notas fiscais entre {PARAM_DATA_INICIO.date()} e {PARAM_DATA_FIM.date()}...")

# Chamar a função 'gerar_vendas' corrigida
df_vendas = gerar_vendas(
    num_notas=PARAM_NUM_NOTAS,
    data_inicio=PARAM_DATA_INICIO,
    data_fim=PARAM_DATA_FIM,
    df_produtos=df_produtos,
    df_filiais=df_filiais,
    df_clientes=df_clientes
)

# Salvar o arquivo de vendas
print(f"Salvando arquivo 'vendas.csv' no Google Drive em {DRIVE_PATH}...")
df_vendas.to_csv(f'{DRIVE_PATH}/vendas.csv', index=False, encoding='utf-8-sig')

Iniciando a geração de 150000 notas fiscais entre 2022-01-01 e 2026-01-01...
Iniciando geração de 150000 notas fiscais (com múltiplos itens)...
Gerando cabeçalhos de 150000 notas...
Total de 374440 linhas de itens geradas.
Enriquecendo linhas com produtos e preços...
Tabela de Vendas gerada com sucesso.
Salvando arquivo 'vendas.csv' no Google Drive em /content/drive/MyDrive/lab_datascience...


In [ ]:
df_vendas.head()

,NUM_NOTA,SERIE_NOTA,DATA_NOTA,COD_FILIAL,COD_CLIENTE,COD_EAN,QUANTIDADE,VALOR_LIQUIDO_lista,DESCONTO,VALOR_LIQUIDO,IMPOSTOS,VALOR_BRUTO
0,100001,001,2023-11-22,LOJA_002,CLI_02008,7893228614771,1,39.26,10.42,28.84,5.77,34.61
1,100001,001,2023-11-22,LOJA_002,CLI_02008,7899186534426,2,11.31,2.46,20.16,4.03,24.19
2,100002,001,2024-11-21,LOJA_002,CLI_01215,7897386441852,1,26.51,6.48,20.03,4.01,24.04
3,100002,001,2024-11-21,LOJA_002,CLI_01215,7894752999404,1,29.52,4.27,25.25,5.05,30.30
4,100002,001,2024-11-21,LOJA_002,CLI_01215,7894857232059,1,20.37,0.00,20.37,4.07,24.44


## **CONCLUSÃO DO PROJETO DE GERAÇÃO DE DADOS**
Neste notebook, executamos um processo completo de Engenharia de Dados Sintéticos para criar um ecossistema de dados coeso e realista para a empresa fictícia, Amana Beauty Lab. O objetivo foi construir uma massa de dados robusta, pronta para servir como base para projetos complexos de Business Intelligence e Data Science.

*  O que Realizamos:

Ambiente: Configuramos o ambiente no Google Colab, instalamos as bibliotecas necessárias (Faker) e autorizamos o acesso ao Google Drive para persistência dos dados.

*  Geração de Entidades:

Criamos e salvamos cinco tabelas (arquivos .csv) que representam as principais entidades de um negócio de varejo:

produtos.csv: Um catálogo de 2.000 SKUs (produtos), cada um com EAN, categoria, custos, preços (líquido e bruto) e um perfil de tier (Alto/Médio Padrão) baseado nas linhas da Amana Beauty Lab.

filiais.csv: Uma rede de lojas físicas e um e-commerce, cada filial com um perfil de público (Alto, Médio, Baixo Padrão) que influencia seu mix de vendas.

clientes.csv: Uma base de 3.000 clientes, cada um com um perfil de compra comportamental (ex: Premium, Caçador de Ofertas) para simular afinidades.

estoque.csv: Um "snapshot" (foto) dos níveis de estoque (mínimo, máximo, atual) para cada produto em cada filial, essencial para análises de abastecimento.

*  Geração de Transações (Vendas):

Criamos a tabela vendas.csv, o coração do dataset. Esta tabela contém centenas de milhares de linhas de itens de nota fiscal, simulando 4 anos de transações (de 2022 a 2026).

*  Principais Características dos Dados Gerados:

Realismo: Os dados não são apenas aleatórios; eles seguem regras de negócio, como:

*  Sazonalidade:

Picos de vendas em datas comemorativas (Natal, Black Friday, Dia das Mães).

*  Afinidade:

Lojas de "Alto Padrão" e clientes "Premium" têm maior probabilidade de comprar produtos de "Alto Padrão".

*  Descontos:

A política de descontos é mais agressiva para clientes "Caçadores de Ofertas" e durante a Black Friday.

*  Integridade:

As tabelas são interligadas por chaves (ex: COD_CLIENTE, COD_FILIAL, COD_EAN), permitindo a realização de JOINS e a criação de um modelo de dados analítico (como um Star Schema).

*  Riqueza Analítica:

A tabela de vendas final inclui métricas cruciais para análise, como VALOR_LIQUIDO_lista (preço de tabela), DESCONTO (valor monetário), VALOR_LIQUIDO (pós-desconto) e IMPOSTOS.


🚀 Próximos Passos:

*  O que Podemos Fazer Agora?

Com este conjunto de dados salvo em seu Google Drive, estamos prontos para iniciar uma variedade de análises avançadas, como:

*  Segmentação de Clientes:

Aplicar um modelo RFV (Recência, Frequência, Valor) para clusterizar clientes e validar os perfis que criamos.

*  Previsão de Demanda:

Usar modelos de Time Series (como SARIMAX ou Prophet) para prever as vendas futuras por loja ou categoria.

*  Análise de Cesta de Compras (Market Basket Analysis):

Descobrir quais produtos são frequentemente comprados juntos.

*  Otimização de Estoque:

Identificar produtos com risco de ruptura ou excesso de estoque.

*  Análise de Elasticidade de Preço:

Entender o impacto real dos descontos no volume de vendas e na margem.